## Introduction: the xarray-simlab modeling framework

Let's briefly review:

- this blog post: https://medium.com/pangeo/pangeo-data-and-models-280b251ff0cd
- xarray-simlab's documentation about the concepts behind the framework: https://xarray-simlab.readthedocs.io/en/latest/framework.html


## Showcase

A brief demonstration (hopefully) showing the interactivity of xarray-simlab (and fastscape).

In [ ]:
import numpy as np
import xsimlab as xs

%load_ext xsimlab.ipython

In [ ]:
# import a xarray-simlab model from fastscape
from fastscape.models import basic_model

# import a model component (process) from fastscape
from fastscape.processes import MultipleFlowRouter

# create a custom model with an alternative set of processes
model = basic_model.update_processes({'flow': MultipleFlowRouter})


In [ ]:
# pre-define some inputs
grid_shape = [201, 201]
grid_length = [2e4, 2e4]
steps = np.arange(0.0, 2e5, 2e3)

u_rate = 1e-3
k_f = 2e-4
k_d = 0.01

In [ ]:
# %create_setup model -d -v
import xsimlab as xs

ds_in = xs.create_setup(
    model=model,
    clocks={'time': steps},
    input_vars={
        # nb. of grid nodes in (y, x)
        'grid__shape': grid_shape,
        # total grid length in (y, x)
        'grid__length': grid_length,
        # node status at borders
        'boundary__status': 'fixed_value',
        # uplift rate
        'uplift__rate': u_rate,
        # random seed
        'init_topography__seed': None,
        # diffusivity (transport coefficient)
        'diffusion__diffusivity': k_d,
        # MFD partioner slope exponent
        'flow__slope_exp': 0.0,
        # bedrock channel incision coefficient
        'spl__k_coef': k_f,
        # drainage area exponent
        'spl__area_exp': 0.4,
        # slope exponent
        'spl__slope_exp': 1,
    },
    output_vars={
        'topography__elevation': 'time',
        'drainage__area': 'time',
        'erosion__rate': 'time',
    }
)


In [ ]:
# run the model
with model, xs.monitoring.ProgressBar():
    ds_out = ds_in.xsimlab.run()
    
# or run a batch of models
#with model:
#    ds_out = ds_in.xsimlab.run(batch_dim='batch', parallel=True, scheduler='processes', store='run.zarr')

In [ ]:
ds_out

In [ ]:
from ipyfastscape import TopoViz3d

app = TopoViz3d(ds_out, time_dim='time')

app.components['vertical_exaggeration'].set_factor(12)

app.show()

In [ ]:
# show topography

app.components['coloring'].set_color_var('topography__elevation')
app.components['coloring'].set_colormap('Cividis')
app.components['coloring'].set_color_scale(log=False)

In [ ]:
# show drainage area

app.components['coloring'].set_color_var('drainage__area')
app.components['coloring'].set_colormap('Blues')
app.components['coloring'].set_color_scale(log=True)

In [ ]:
# show erosion rate

app.components['coloring'].set_color_var('erosion__rate')
app.components['coloring'].set_colormap('RdYlBu')
app.components['coloring'].set_color_limits(-2e-3, 2e-3)
app.components['coloring'].set_color_scale(log=False)